<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [3]:
!pip install plotly

In [58]:
#Start Here
import pandas as pd
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore


from ast import literal_eval
import spacy
from spacy.tokenizer import Tokenizer

import tqdm
from tqdm import tqdm

import pyLDAvis.gensim
import seaborn as sns
import matplotlib.pyplot as plt

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [51]:
df = pd.read_csv('./data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')
print(df.shape, "\n")
df.head(2)

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(28332, 24) 



,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."


In [53]:
df['reviews.text'][0]

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'I order 3 of them and one of the item is bad quality. Is missing backup spring so I have to put a pcs of aluminum to make the battery work.'

In [59]:
nlp = spacy.load('en_core_web_lg')
tokenizer = Tokenizer(nlp.vocab)

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [60]:
# df['Description'] = df['Description'].astype(str)
df['reviews.text'] = df['reviews.text'].str.strip()
df['reviews.text'] = df['reviews.text'].replace({",":"", "'":"", "''":"", ".":""})

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [61]:
df['reviews.text']

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0        I order 3 of them and one of the item is bad q...
1        Bulk is always the less expensive way to go fo...
2        Well they are not Duracell but for the price i...
3        Seem to work as well as name brand batteries a...
4        These batteries are very long lasting the pric...
                               ...                        
28327    I got 2 of these for my 8 yr old twins. My 11 ...
28328    I bought this for my niece for a Christmas gif...
28329    Very nice for light internet browsing, keeping...
28330    This Tablet does absolutely everything I want!...
28331    At ninety dollars, the expectionations are low...
Name: reviews.text, Length: 28332, dtype: object

In [62]:
def get_lemmas(text):
    lemmas = []
    # these are repeated words
    STOP_WORDS = nlp.Defaults.stop_words.union(['english', 'amazon',
                                               ])
    
    doc = nlp(text)
    
    for token in doc:
        if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos != 'PRON') \
                and (token.is_space == False) and (token.is_digit == False) and (token.text.lower() not in STOP_WORDS):
                      lemmas.append(token.lemma_.lower())
    return lemmas

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [63]:
df['reviews.text'] = df['reviews.text'].apply(get_lemmas)

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [66]:
df['reviews.text'][5]

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['lot',
 'battery',
 'christmas',
 'amazonbasics',
 'cell',
 'good',
 'notice',
 'difference',
 'brand',
 'battery',
 'amazon',
 'basic',
 'brand',
 'lot',
 'easy',
 'purchase',
 'arrive',
 'house',
 'hand',
 'buy']

In [75]:
df['reviews.text'].head()

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    [item, bad, quality, miss, backup, spring, pc,...
1                      [expensive, way, product, like]
2                             [duracell, price, happy]
3                  [work, brand, battery, well, price]
4                           [long, last, price, great]
Name: reviews.text, dtype: object

In [76]:
# Dict representation of all our keywords
id2word = corpora.Dictionary(df['reviews.text'])

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [77]:
id2word.token2id['batteries']

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


199

In [78]:
len(id2word.keys())

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


8550

In [79]:
id2word.filter_extremes(no_below=15, no_above=0.85)

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [80]:
len(id2word.keys())

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1501

In [81]:
corpus = [id2word.doc2bow(text) for text in df['reviews.text']]

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [83]:
corpus[900]

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(38, 1), (46, 1), (179, 1), (195, 1), (295, 1), (681, 1)]

In [85]:
id2word[78]

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'comment'

In [86]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   num_topics=20,
                   passes=50, 
                   workers=12
                  )

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [87]:
lda.print_topics()

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.106*"battery" + 0.022*"amazon" + 0.017*"buy" + 0.016*"use" + 0.015*"work" + 0.013*"brand" + 0.013*"remote" + 0.011*"box" + 0.011*"aa" + 0.011*"toy"'),
 (1,
  '0.064*"ipad" + 0.055*"tablet" + 0.035*"app" + 0.027*"need" + 0.024*"amazon" + 0.020*"trip" + 0.018*"little" + 0.018*"android" + 0.014*"device" + 0.012*"mini"'),
 (2,
  '0.099*"read" + 0.080*"book" + 0.048*"kindle" + 0.044*"light" + 0.039*"reader" + 0.028*"easy" + 0.021*"e" + 0.018*"like" + 0.015*"size" + 0.013*"love"'),
 (3,
  '0.037*"app" + 0.030*"device" + 0.024*"screen" + 0.020*"wifi" + 0.018*"home" + 0.018*"tablet" + 0.015*"thing" + 0.015*"work" + 0.014*"button" + 0.014*"download"'),
 (4,
  '0.119*"kindle" + 0.091*"fire" + 0.038*"screen" + 0.031*"hd" + 0.030*"new" + 0.025*"well" + 0.022*"great" + 0.019*"model" + 0.019*"version" + 0.018*"like"'),
 (5,
  '0.048*"app" + 0.040*"google" + 0.038*"amazon" + 0.035*"play" + 0.031*"good" + 0.030*"store" + 0.028*"tablet" + 0.025*"expect" + 0.017*"price" + 0.017*"video"'),
 (6,

In [88]:
words = [re.findall(r'"[^"]*"', t[1]) for t in lda.print_topics(20)]

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [89]:
topic_labels = [', '.join(t[0:5]) for t in words]

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [93]:
for id, t in enumerate(topic_labels): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
"battery", "amazon", "buy", "use", "work"

------ Topic 1 ------
"ipad", "tablet", "app", "need", "amazon"

------ Topic 2 ------
"read", "book", "kindle", "light", "reader"

------ Topic 3 ------
"app", "device", "screen", "wifi", "home"

------ Topic 4 ------
"kindle", "fire", "screen", "hd", "new"

------ Topic 5 ------
"app", "google", "amazon", "play", "good"

------ Topic 6 ------
"kid", "tablet", "child", "great", "love"

------ Topic 7 ------
"good", "price", "tablet", "quality", "money"

------ Topic 8 ------
"love", "gift", "christmas", "tablet", "great"

------ Topic 9 ------
"charge", "problem", "time", "buy", "break"

------ Topic 10 ------
"great", "price", "product", "recommend", "work"

------ Topic 11 ------
"old", "year", "love", "son", "tablet"

------ Topic 12 ------
"tablet", "nice", "great", "storage", "fast"

------ Topic 13 ------
"buy", "good", "thank", "need", "product"

------ Topic 14 ------
"game", "play", "love", "watch", "movie"

---

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [94]:
pyLDAvis.enable_notebook()

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [95]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.191106  0.129967       1        1  15.375137
6      0.159476 -0.008649       2        1   7.541356
16    -0.276434  0.177818       3        1   6.759451
2      0.134676  0.009383       4        1   5.521235
17     0.175831 -0.108599       5        1   5.500759
10    -0.176479  0.019962       6        1   5.145629
4     -0.014236 -0.056083       7        1   5.138543
3     -0.025962 -0.091998       8        1   4.754666
7     -0.158783 -0.102601       9        1   4.604614
11     0.220180  0.124026      10        1   4.524179
5     -0.071180 -0.119776      11        1   4.015865
13    -0.076530  0.130341      12        1   3.975396
14     0.257560  0.000670      13        1   3.862189
19    -0.006940 -0.077392      14        1   3.638023
12    -0.038616 -0.197990      15        1   3.535487
1     -0.030794 -0.134640      16        1   3.404905
8      0.198345  0.191985      17        1   3.396134
15    -0.070675  0.061396      18        1   3.121246
9      0.012134  0.190647      19        1   3.106186
18    -0.020468 -0.138467      20        1   3.079000, topic_info=         Term         Freq        Total Category  logprob  loglift
2     battery  8010.000000  8010.000000  Default  30.0000  30.0000
73       love  4884.000000  4884.000000  Default  29.0000  29.0000
396       old  2436.000000  2436.000000  Default  28.0000  28.0000
84       year  2179.000000  2179.000000  Default  27.0000  27.0000
19       long  2521.000000  2521.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
65        get   105.385202  1354.560170  Topic20  -4.3792   0.9270
1111  feature    73.727859   772.064640  Topic20  -4.7364   1.1319
333   awesome    62.776343   417.068327  Topic20  -4.8972   1.5869
6     quality    65.204598  1085.051184  Topic20  -4.8593   0.6687
24        buy    64.236449  3855.712086  Topic20  -4.8742  -0.6142

[1169 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1351      7  0.096432    $
1351     10  0.002296    $
1351     11  0.183680    $
1351     12  0.034440    $
1351     14  0.036736    $
...     ...       ...  ...
806       2  0.020758   yr
806      10  0.851062   yr
806      13  0.128697   yr
1438      2  0.486262  yrs
1438     17  0.486262  yrs

[3621 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 17, 3, 18, 11, 5, 4, 8, 12, 6, 14, 15, 20, 13, 2, 9, 16, 10, 19])

In [96]:
distro = [lda[doc] for doc in corpus]

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [98]:
distro[0]

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0, 0.5359023), (3, 0.2321609), (7, 0.13747545)]

In [99]:
def update(doc):
    d_dist = {k:0 for k in range(0,20)}
    for t in doc:
        d_dist[t[0]] = t[1]
    return d_dist

new_distro = [update(doc) for doc in distro] 

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [100]:
topics = pd.DataFrame.from_records(new_distro)
topics.columns = topic_labels

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [101]:
topics.head()

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,"""battery"", ""amazon"", ""buy"", ""use"", ""work""","""ipad"", ""tablet"", ""app"", ""need"", ""amazon""","""read"", ""book"", ""kindle"", ""light"", ""reader""","""app"", ""device"", ""screen"", ""wifi"", ""home""","""kindle"", ""fire"", ""screen"", ""hd"", ""new""","""app"", ""google"", ""amazon"", ""play"", ""good""","""kid"", ""tablet"", ""child"", ""great"", ""love""","""good"", ""price"", ""tablet"", ""quality"", ""money""","""love"", ""gift"", ""christmas"", ""tablet"", ""great""","""charge"", ""problem"", ""time"", ""buy"", ""break""","""great"", ""price"", ""product"", ""recommend"", ""work""","""old"", ""year"", ""love"", ""son"", ""tablet""","""tablet"", ""nice"", ""great"", ""storage"", ""fast""","""buy"", ""good"", ""thank"", ""need"", ""product""","""game"", ""play"", ""love"", ""watch"", ""movie""","""fast"", ""good"", ""work"", ""purchase"", ""pleased""","""battery"", ""long"", ""brand"", ""life"", ""good""","""use"", ""easy"", ""tablet"", ""great"", ""read""","""tablet"", ""great"", ""price"", ""amazon"", ""black""","""tablet"", ""amazon"", ""card"", ""memory"", ""account"""
0,0.535902,0.0000,0.0000,0.232161,0.0000,0.0000,0.0000,0.137475,0.000000,0.0000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000,0.0000,0.0000,0.0000
1,0.010000,0.0100,0.0100,0.010000,0.0100,0.0100,0.0100,0.010000,0.010000,0.0100,0.248612,0.0100,0.0100,0.0100,0.0100,0.0100,0.571384,0.0100,0.0100,0.0100
2,0.012500,0.0125,0.0125,0.012500,0.0125,0.0125,0.0125,0.012500,0.164172,0.0125,0.012500,0.0125,0.0125,0.0125,0.0125,0.0125,0.610824,0.0125,0.0125,0.0125
3,0.000000,0.0000,0.0000,0.000000,0.0000,0.0000,0.0000,0.000000,0.000000,0.0000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.841666,0.0000,0.0000,0.0000
4,0.010000,0.0100,0.0100,0.010000,0.0100,0.0100,0.0100,0.010000,0.010000,0.0100,0.250004,0.0100,0.0100,0.0100,0.0100,0.0100,0.569995,0.0100,0.0100,0.0100


In [102]:
df['primary_topic'] = topics.idxmax(axis=1)

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [103]:
df.head()

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs,primary_topic
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,"[item, bad, quality, miss, backup, spring, pc,...",... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht...","""battery"", ""amazon"", ""buy"", ""use"", ""work"""
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,"[expensive, way, product, like]",... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht...","""battery"", ""long"", ""brand"", ""life"", ""good"""
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,"[duracell, price, happy]",... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht...","""battery"", ""long"", ""brand"", ""life"", ""good"""
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,"[work, brand, battery, well, price]",... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht...","""battery"", ""long"", ""brand"", ""life"", ""good"""
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,"[long, last, price, great]",... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht...","""battery"", ""long"", ""brand"", ""life"", ""good"""


In [104]:
df['primary_topic'].value_counts()

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


"battery", "amazon", "buy", "use", "work"           5694
"battery", "long", "brand", "life", "good"          3155
"great", "price", "product", "recommend", "work"    2216
"kid", "tablet", "child", "great", "love"           2048
"use", "easy", "tablet", "great", "read"            1607
"good", "price", "tablet", "quality", "money"       1571
"read", "book", "kindle", "light", "reader"         1239
"love", "gift", "christmas", "tablet", "great"      1228
"kindle", "fire", "screen", "hd", "new"             1115
"old", "year", "love", "son", "tablet"              1074
"game", "play", "love", "watch", "movie"             951
"buy", "good", "thank", "need", "product"            826
"app", "google", "amazon", "play", "good"            808
"fast", "good", "work", "purchase", "pleased"        767
"app", "device", "screen", "wifi", "home"            717
"tablet", "great", "price", "amazon", "black"        708
"tablet", "nice", "great", "storage", "fast"         706
"tablet", "amazon", "card", "me

In [ ]:
# 'battery', 'tablet', 'amazon', 'good', 'great', 'app', 'buy', 'love'

In [6]:
agg = pd.pivot_table(df, values=['reviews.text'], 
                     index=['primary_topic'], 
                     columns=['reviews.rating'], aggfunc='count')

NameError: name 'pd' is not defined

In [108]:
agg = agg.reset_index()

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [110]:
agg.head()

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


primary_topic reviews.text  \
reviews.rating                                                        1   
0                "app", "device", "screen", "wifi", "home"         35.0   
1                "app", "google", "amazon", "play", "good"         25.0   
2                "battery", "amazon", "buy", "use", "work"        562.0   
3               "battery", "long", "brand", "life", "good"         80.0   
4                "buy", "good", "thank", "need", "product"         33.0   

                                             
reviews.rating      2      3      4       5  
0                36.0   65.0  278.0   303.0  
1                14.0  102.0  283.0   384.0  
2               263.0  279.0  629.0  3961.0  
3                94.0  179.0  433.0  2369.0  
4                25.0   46.0  126.0   596.0

In [113]:
!pip install altair

C:\Users\lomopa\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import altair as alt
from plotly.offline import iplot
import plotly.graph_objs as go
import IPython
import plotly.offline as py
from altair import datum
import plotly.express as px
import plotly

In [ ]:
# source = new_df

# alt.Chart(source).transform_fold(
#     ['Nozzle_Bed_Temp', 'Print_Fan_Speed', 'infill_roughness'],
#     as_=['Experiment', 'Measurement']
# ).mark_area(
#     opacity=0.3,
#     interpolate='step'
# ).encode(
#     alt.X('Measurement:Q', bin=alt.Bin(maxbins=100)),
#     alt.Y('count()', stack=None),
#     alt.Color('Experiment:N')
# )

In [ ]:
fig = px.scatter_3d(new_df, x='Nozzle_Bed_Temp', y='Print_Fan_Speed', z='infill_roughness',
              color='Print_Fan_Speed')
fig.show()

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling